In [1]:
import os
import sys
import typing
import json
import pandas as pd
if 'NOTEBOOKDIR' not in locals():
    NOTEBOOKDIR = os.getcwd()
from pathlib import Path
RUNDIR = str(Path(NOTEBOOKDIR).parent)
sys.path.append(str(RUNDIR))
import ray

In [2]:
# run "ray start --head" from the repo root directory from within the venv lme.
# If you to attach another machine to the cluster, then run "ray start --address=<head-node-ip>:6379" there.
ray.init(address='auto')

2022-09-02 05:15:10,955	INFO worker.py:1333 -- Connecting to existing Ray cluster at address: 10.2.143.243:6379...
2022-09-02 05:15:11,003	INFO worker.py:1509 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.0.0
Dashboard:,http://127.0.0.1:8265


In [3]:
num_fewshots = [0, 5, 25]
task_models = [('webqs_dg', 'dist_gen'), ('hellaswag_dg', 'dist_gen')]
pretrained = [None, 'EleutherAI/gpt-neo-1.3B']

@ray.remote(max_calls=1, num_gpus=1)
def run_eval(args):
    from main import main
    return main(*args)

futures = []
for num_fewshot in num_fewshots:
    for task, model in task_models:
        for submodel in pretrained:
            _args = [
                "--device", "0",
                "--output_dir", "lmeval_results/",
                "--tasks", task,
                "--model", model,
                "--no_cache",
                '--num_fewshot', f'{num_fewshot}',
                '--task_args', f'encoding_scheme=cross_encoding',
            ]
            if submodel is not None:
                _args.extend(['--model_args', f'pretrained={submodel}'])
            future = run_eval.remote(_args)
            futures.append(future)

# for num_fewshot in num_fewshots:
#     _args = [
#     "--device", "0",
#     "--output_dir", "lmeval_results/",
#     "--tasks", "webqs",
#     "--model", "gpt2",
#     "--num_fewshot", f'{num_fewshot}',
#     '--model_args', 'pretrained=EleutherAI/gpt-neo-1.3B'
#     ]
#     future = run_eval.remote(_args)
#     futures.append(future)

results = ray.get(futures)

(run_eval pid=1854674) 2022-09-02 05:15:13.430067: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(run_eval pid=1854679) 2022-09-02 05:15:13.459775: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(run_eval pid=1854676) 2022-09-02 05:15:13.430068: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(run_eval pid=1854681) 2022-09-02 05:15:13.580462

(run_eval pid=1854674) Selected Tasks: ['webqs_dg']
(run_eval pid=1854674) Using device '0'
(run_eval pid=1854679) Selected Tasks: ['hellaswag']
(run_eval pid=1854679) Using device '0'
(run_eval pid=1854676) Selected Tasks: ['webqs']
(run_eval pid=1854676) Using device '0'
(run_eval pid=1854682) Selected Tasks: ['hellaswag']
(run_eval pid=1854682) Using device '0'
(run_eval pid=1854678) Selected Tasks: ['webqs_dg']
(run_eval pid=1854678) Using device '0'
(run_eval pid=1854677) Selected Tasks: ['hellaswag_dg']
(run_eval pid=1854677) Using device '0'
(run_eval pid=1854675) Selected Tasks: ['hellaswag_dg']
(run_eval pid=1854675) Using device '0'
(run_eval pid=1854681) Selected Tasks: ['webqs']
(run_eval pid=1854681) Using device '0'


(run_eval pid=1854674) Using custom data configuration default
(run_eval pid=1854674) Reusing dataset web_questions (/home/ssingh/.cache/huggingface/datasets/web_questions/default/1.0.0/e6742cc64f6652db0c52cb07b5414e3c001512bf5bde7aa5587353c31db1ed8b)
100%|██████████| 2/2 [00:00<00:00, 865.52it/s]


RayTaskError(TypeError): [36mray::run_eval()[39m (pid=1854682, ip=10.2.143.243)
  File "/tmp/ipykernel_1854556/3269644108.py", line 8, in run_eval
  File "/home/ssingh/src/lm_eval_harness/main.py", line 81, in main
    results = evaluator.simple_evaluate(
  File "/home/ssingh/src/lm_eval_harness/lm_eval/utils.py", line 162, in _wrapper
    return fn(*args, **kwargs)
  File "/home/ssingh/src/lm_eval_harness/lm_eval/evaluator.py", line 85, in simple_evaluate
    task_dict = lm_eval.tasks.get_task_dict(tasks, task_args)
  File "/home/ssingh/src/lm_eval_harness/lm_eval/tasks/__init__.py", line 322, in get_task_dict
    task_name_dict = {
  File "/home/ssingh/src/lm_eval_harness/lm_eval/tasks/__init__.py", line 323, in <dictcomp>
    task_name: get_task(task_name)(**utils.simple_parse_args_string(task_args))
TypeError: __init__() got an unexpected keyword argument 'encoding_scheme'

(run_eval pid=1854675) Using custom data configuration default
(run_eval pid=1854675) Reusing dataset hellaswag (/home/ssingh/.cache/huggingface/datasets/hellaswag/default/0.1.0/c8c5bc30147e6345a39bfabe8856829801d4db7beb0271e44021811974fac112)
100%|██████████| 3/3 [00:00<00:00, 543.87it/s]


In [ ]:
print('Done')

(run_eval pid=1854674) Running distributed_encoding_generation requests


  2%|▏         | 43/2032 [00:01<00:36, 54.76it/s]
(run_eval pid=1855809) 2022-09-02 05:15:26.025339: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
  3%|▎         | 52/2032 [00:01<00:35, 55.27it/s]
(run_eval pid=1855875) 2022-09-02 05:15:26.206744: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
  0%|          | 0/10042 [00:00<?, ?it/s]


(run_eval pid=1854675) Running distributed_encoding_generation requests


  0%|          | 22/10042 [00:01<06:30, 25.64it/s]


(run_eval pid=1855809) Selected Tasks: ['webqs_dg']
(run_eval pid=1855809) Using device '0'


...

 21%|██        | 422/2032 [00:21<01:43, 15.60it/s]
(run_eval pid=1860020) Using custom data configuration default
(run_eval pid=1860020) Reusing dataset hellaswag (/home/ssingh/.cache/huggingface/datasets/hellaswag/default/0.1.0/c8c5bc30147e6345a39bfabe8856829801d4db7beb0271e44021811974fac112)
  0%|          | 0/10042 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (2000 > 1024). Running this sequence through the model will result in indexing errors
(run_eval pid=1860020) Sequence of length 2055 will be truncated to 1024
 28%|██▊       | 577/2032 [00:30<00:56, 25.90it/s]


(run_eval pid=1860020) Running distributed_encoding_generation requests


 11%|█▏        | 1154/10042 [01:31<12:09, 12.18it/s]
(run_eval pid=1860020) Sequence of length 2045 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 2074 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 2051 will be truncated to 1024
  0%|          | 1/10042 [00:00<1:11:05,  2.35it/s]Sequence of length 1901 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 1903 will be truncated to 1024
 29%|██▉       | 589/2032 [00:30<00:57, 24.98it/s]
(run_eval pid=1860020) Sequence of length 1899 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 1907 will be truncated to 1024
 12%|█▏        | 1160/10042 [01:32<11:14, 13.17it/s]
(run_eval pid=1860020) Sequence of length 2005 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 2012 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 2010 will be truncated to 1024
  0%|          | 3/10042 [00:00<39:29,  4.24it/s]Sequence of length 2132 will be trun

(run_eval pid=1859715) Running distributed_encoding_generation requests


 49%|████▉     | 996/2032 [01:42<02:58,  5.79it/s]
(run_eval pid=1860020) Sequence of length 2232 will be truncated to 1024
 32%|███▏      | 647/2032 [00:33<01:21, 17.09it/s]
(run_eval pid=1860020) Sequence of length 2035 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 2041 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 2040 will be truncated to 1024
  0%|          | 18/10042 [00:03<30:25,  5.49it/s]Sequence of length 1789 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 1786 will be truncated to 1024
  0%|          | 2/2032 [00:00<08:09,  4.14it/s]
(run_eval pid=1860020) Sequence of length 1802 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 1790 will be truncated to 1024
  0%|          | 3/2032 [00:00<06:43,  5.02it/s]
(run_eval pid=1860020) Sequence of length 1917 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 1913 will be truncated to 1024
(run_eval pid=1860020) Sequence of length 1